# Dataset weights

Notebook to get the weight of the different classes present in the dataset.

This is computed only in the trainingset.

In [2]:
from sflizard import LizardDataModule

from tqdm import tqdm
import torch

warnings.py (30): You want to use `gym` which is not installed yet, install it with `pip install gym`.


In [6]:
IMG_SIZE=540
DATA_PATH="../data_shuffle_train.pkl"
TEST_DATA_PATH="../data_shuffle_test.pkl"
SEED=303
BATCH_SIZE=4
N_RAYS = 32

ANNOTATION_TARGET = "stardist_class"

In [7]:
aditional_args={"n_rays":N_RAYS}

# create the datamodule
dm = LizardDataModule(
    train_data_path=DATA_PATH,
    test_data_path=TEST_DATA_PATH,
    annotation_target=ANNOTATION_TARGET,
    batch_size=BATCH_SIZE,
    num_workers = 1,
    input_size=IMG_SIZE,
    seed=SEED,
    aditional_args=aditional_args,
)
dm.setup()

Training with 2323 examples


In [8]:
classes_representation = {}
for i in range(7):
    classes_representation[i] = 0
train_iter = iter(dm.train_dataloader())
for b in tqdm(range(len(train_iter))):
    image, obj_probabilities, distances, classes = next(train_iter)
    for i in range(len(image)):
        class_map = torch.flatten(classes[i].int())
        occurences = torch.bincount(class_map)
        for o in range(len(occurences)):
            classes_representation[o] += occurences[o]

100%|██████████| 581/581 [11:47<00:00,  1.22s/it]


In [9]:
print(classes_representation)

{0: tensor(572276694), 1: tensor(1003007), 2: tensor(64618826), 3: tensor(12706493), 4: tensor(4096427), 5: tensor(857558), 6: tensor(21827795)}


In [10]:
total = 0
for c in classes_representation.keys():
    classes_representation[c] = classes_representation[c].item()
    total += classes_representation[c]
for c in classes_representation.keys():
    classes_representation[c] /= total
print(classes_representation)

{0: 0.8448300055448379, 1: 0.0014807005391897215, 2: 0.09539427990034645, 3: 0.01875810541333253, 4: 0.006047397144438008, 5: 0.0012659797917526589, 6: 0.032223531666102735}


In [19]:
print(classes_representation)

{0: 0.822068738042566, 1: 0.0015632805252265753, 2: 0.13668895606007572, 3: 0.01808995955003234, 4: 0.006081943184276731, 5: 0.0005239083549402539, 6: 0.014983214282882448}


In [ ]:
{0: 0.8435234983048621, 1: 0.0015844697497448515, 2: 0.09702835179125052, 3: 0.018770678077839286, 4: 0.005716505874930195, 5: 0.0011799091886332306, 6: 0.03219658701273987}